# **MODEL BUILDING**

## **4.Reinforcement Learning Model for Cyclone and Flood**

#*Import Dependencies*

In [ ]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os


In [ ]:
pip install stable_baselines3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# *Building an environment*

In [ ]:
class Water_DisasterEnv(Env):
    def __init__(self):
        # Actions we can take: closed room, stay, vacate
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start rainfall
        self.state = 10 + random.randint(-3,3)
        # Set decision duration
        self.decision_duration = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 rainfall
        # 1 -1 = 0 
        # 2 -1 = 1 rainfall 
        self.state += action -1 
        # Reduce decision duration by 1 minute
        self.decision_duration -= 1 
        
        # Calculate reward
        if self.state >=10 and self.state <=20: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if time is over
        if self.decision_duration <= 0: 
            done = True
        else:
            done = False
        
        # Apply rainfall fluctuations
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset rainfall
        self.state = np.array([15 + random.randint(-3,3)]).astype(float)
        # Reset decision duration
        self.decision_duration = 60 
        return self.state

In [ ]:
env = Water_DisasterEnv()

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  -self.np_random.exponential(size=upp_bounded[upp_bounded].shape)


In [ ]:
env.observation_space.sample()

array([68.27607], dtype=float32)

# *Test Environment*

In [ ]:
episodes = 100
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-14
Episode:2 Score:60
Episode:3 Score:16
Episode:4 Score:32
Episode:5 Score:-4
Episode:6 Score:2
Episode:7 Score:-26
Episode:8 Score:30
Episode:9 Score:50
Episode:10 Score:24
Episode:11 Score:26
Episode:12 Score:52
Episode:13 Score:34
Episode:14 Score:60
Episode:15 Score:28
Episode:16 Score:60
Episode:17 Score:30
Episode:18 Score:22
Episode:19 Score:6
Episode:20 Score:58
Episode:21 Score:38
Episode:22 Score:42
Episode:23 Score:28
Episode:24 Score:20
Episode:25 Score:60
Episode:26 Score:32
Episode:27 Score:46
Episode:28 Score:-14
Episode:29 Score:60
Episode:30 Score:-44
Episode:31 Score:46
Episode:32 Score:20
Episode:33 Score:38
Episode:34 Score:36
Episode:35 Score:56
Episode:36 Score:60
Episode:37 Score:-20
Episode:38 Score:56
Episode:39 Score:52
Episode:40 Score:-48
Episode:41 Score:44
Episode:42 Score:-6
Episode:43 Score:60
Episode:44 Score:58
Episode:45 Score:60
Episode:46 Score:6
Episode:47 Score:46
Episode:48 Score:50
Episode:49 Score:42
Episode:50 Score:60
Episod

# *Train Model*

In [ ]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  def reset(self, **kwargs):


In [ ]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -60      |
| time/              |          |
|    fps             | 426      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -60          |
| time/                   |              |
|    fps                  | 549          |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0056281523 |
|    clip_fraction        | 0.0476       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   |

# *Save Model*

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, env, n_eval_episodes=100)

(-49.6, 3.730951621235526)